In [1]:
from datetime import date, datetime

import numpy as np
import pandas as pd

In [2]:
# Using low_memory to be able load mixed types without setting data type for each column separately
df = pd.read_csv('data/Ajoneuvorekisteri.csv', sep=';', encoding='ISO-8859-1', low_memory=False)

In [3]:
# Checking how the dataframe looks like
df.head()

,ajoneuvoluokka,ensirekisterointipvm,ajoneuvoryhma,ajoneuvonkaytto,variantti,versio,kayttoonottopvm,vari,ovienLukumaara,korityyppi,...,tyyppihyvaksyntanro,yksittaisKayttovoima,kunta,NEDC_Co2,NEDC2_Co2,WLTP_Co2,WLTP2_Co2,matkamittarilukema,valmistenumero2,jarnro
0,MUU,NaN,21.0,1.0,NaN,NaN,19670000,NaN,NaN,NaN,...,NaN,01,91.0,NaN,NaN,NaN,NaN,NaN,NaN,1
1,MUU,1976-09-01,13.0,1.0,NaN,NaN,19760000,NaN,NaN,NaN,...,NaN,NaN,475.0,NaN,NaN,NaN,NaN,NaN,NaN,2
2,M1,1984-07-09,NaN,1.0,NaN,NaN,19840000,1,NaN,NaN,...,NaN,01,179.0,NaN,NaN,NaN,NaN,NaN,NaN,3
3,MUU,1983-09-22,13.0,1.0,NaN,NaN,19830000,NaN,NaN,NaN,...,NaN,NaN,734.0,NaN,NaN,NaN,NaN,NaN,NaN,4
4,O1,1994-02-09,1.0,1.0,NaN,NaN,19940209,NaN,NaN,NaN,...,NaN,NaN,761.0,NaN,NaN,NaN,NaN,NaN,NaN,5


In [4]:
df.shape

(5163486, 41)

In [5]:
# Create a new dataframe and include passenger cars only
filter_passenger_cars = ['M1', 'M1G']
df = df[df['ajoneuvoluokka'].isin(filter_passenger_cars)]

In [6]:
# Group cars by fuel 
df_fuels = df.groupby(['kayttovoima']).size().nlargest(3).to_frame(name='count').reset_index()
df_fuels = df_fuels.rename(columns={'kayttovoima': 'fuel'})
df_fuels

,fuel,count
0,01,1896836
1,02,655330
2,04,44864


In [7]:
# Replace ids with human readable titles
df_fuels['fuel'] = df_fuels['fuel'].replace({"01": "Petrol", "02": "Diesel fuel", "04": "Electricity"})
df_fuels

,fuel,count
0,Petrol,1896836
1,Diesel fuel,655330
2,Electricity,44864


In [8]:
sum_others = len(df) - df_fuels['count'].sum()
df_fuels_others = pd.DataFrame([['Others', sum_others]],
                               columns=['fuel', 'count'])
df_fuels_others

,fuel,count
0,Others,20674


In [9]:
df_fuels_final = pd.concat([df_fuels, df_fuels_others], ignore_index=True)

In [10]:
df_fuels_final

,fuel,count
0,Petrol,1896836
1,Diesel fuel,655330
2,Electricity,44864
3,Others,20674


In [11]:
# How many of the cars are hybrid. Hybrids are included to the petrol and diesel counts.
# The 'sahkohybridi' is True when the car is hybrid 
df['sahkohybridi'].unique()


array([nan, False, True], dtype=object)

In [12]:
# Replacing NaNs with False
df['sahkohybridi'] = df['sahkohybridi'].fillna(False)

# Setting column type as boolean
df['sahkohybridi'] = df['sahkohybridi'].astype(bool)

In [13]:
df_hybrids = df[df['sahkohybridi'] == True]

# The amount of hybrid cars
len(df_hybrids)

211369

In [14]:
# Let's see how many of the hybrids are plug-in hybrids and how many are self-charging
df_hybrids.groupby(['sahkohybridinluokka']).size()

sahkohybridinluokka
1.0     81407
2.0    129933
3.0         4
4.0         5
dtype: int64

In [15]:
# There are also few fuel cell hybrids (groups 3.0 and 4.0) but let's ignore those and create a new dataframe only from the two major groups
df_hybrids_grouped = df_hybrids.groupby(['sahkohybridinluokka']).size().nlargest(2).to_frame(name='count').reset_index()
# Translate the title
df_hybrids_grouped = df_hybrids_grouped.rename(columns={'sahkohybridinluokka': 'hybrid_type'})
# Set column type as integer
df_hybrids_grouped['hybrid_type'] = df_hybrids_grouped['hybrid_type'].astype(int)
# Replace integer values with english text
df_hybrids_grouped['hybrid_type'] = df_hybrids_grouped['hybrid_type'].replace({1: "Plug-in hybrids", 2: "Self-charging hybrids"})

# Show the results
df_hybrids_grouped

,hybrid_type,count
0,Self-charging hybrids,129933
1,Plug-in hybrids,81407


In [16]:
# Comparing top 5 brands from electric cars and petrol and diesel cars
df_non_electric_cars = df.loc[(df['kayttovoima'].isin(['01', '02']) & df['sahkohybridi'] != True)]
df_electric_cars = df.loc[df['kayttovoima'] == '04']

# First non-electric cars
df_non_electric_cars.groupby('merkkiSelvakielinen').size().nlargest(10).to_frame(name='count').reset_index()

,merkkiSelvakielinen,count
0,Toyota,297263
1,Volvo,193380
2,Ford,173285
3,Mercedes-Benz,165459
4,Volkswagen,165452
5,Skoda,150398
6,Nissan,140247
7,Opel,118556
8,"Volkswagen, VW",110556
9,Audi,108958


In [17]:
# And then electric cars
df_electric_cars.groupby('merkkiSelvakielinen').size().nlargest(10).to_frame(name='count').reset_index()

,merkkiSelvakielinen,count
0,Tesla,9943
1,"Volkswagen, VW",6374
2,Hyundai,3322
3,Nissan,3304
4,Skoda,2270
5,Volvo,2036
6,Mercedes-Benz,2008
7,Kia,1883
8,Audi,1849
9,Tesla Motors,1434


In [23]:
# How many cars was registered in 2022

# Creating new column for registering year
df['year_registered'] = pd.DatetimeIndex(df['ensirekisterointipvm']).year

df.query('year_registered == 2022')



,ajoneuvoluokka,ensirekisterointipvm,ajoneuvoryhma,ajoneuvonkaytto,variantti,versio,kayttoonottopvm,vari,ovienLukumaara,korityyppi,...,kunta,NEDC_Co2,NEDC2_Co2,WLTP_Co2,WLTP2_Co2,matkamittarilukema,valmistenumero2,jarnro,age,year_registered
1602448,M1,2022-04-02,NaN,1.0,AOBA1G,5BLBNT,20080328,8,4.0,AC,...,707.0,189.0,NaN,NaN,NaN,62739.0,WF0GXXGBBG,1602449,0.747449,2022.0
2321049,M1,2022-10-21,NaN,1.0,ACBKPX0,FA6FA62E018ST0GGS,20080317,0,4.0,AC,...,743.0,174.0,NaN,NaN,NaN,203433.0,WVWZZZ3CZ8,2321050,0.194391,2022.0
2432260,M1,2022-10-14,21.0,1.0,NaN,NaN,19790000,1,2.0,AD,...,301.0,NaN,NaN,NaN,NaN,62342.0,NaN,2432261,0.213557,2022.0
2520974,M1,2022-09-08,21.0,1.0,NaN,NaN,19760908,5,4.0,AA,...,564.0,NaN,NaN,NaN,NaN,195401.0,NaN,2520975,0.312121,2022.0
2548167,M1,2022-03-30,21.0,1.0,NaN,NaN,19740000,5,2.0,AE,...,694.0,NaN,NaN,NaN,NaN,56985.0,NaN,2548168,0.755662,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5163472,M1,2022-12-30,NaN,1.0,ZNR20(W),ZNR11L-GWMEKW(1D),20051021,8,5.0,AC,...,908.0,184.0,NaN,NaN,NaN,122364.0,NMTER16RX0,5163473,0.002738,2022.0
5163475,M1,2022-12-30,NaN,1.0,6N71,IAW50700,20210615,9,4.0,AC,...,91.0,NaN,45.0,NaN,42.0,19331.0,WBA6N71020,5163476,0.002738,2022.0
5163478,M1,2022-12-30,NaN,5.0,51ECA0,ZZAL050B,20220519,6,4.0,AA,...,49.0,NaN,NaN,NaN,NaN,24979.0,W1K2951211,5163479,0.002738,2022.0
5163480,M1,2022-12-30,NaN,5.0,Y5LD,BZb3s5T3,20220301,9,5.0,AF,...,680.0,NaN,NaN,NaN,NaN,14197.0,LRWYGCEK8N,5163481,0.002738,2022.0
